<a href="https://colab.research.google.com/github/dw-shin/AK-SSAM/blob/main/projects/project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Convection-Diffusion Equations

In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format='retina'

## 1D

Consider the following convection-diffusion equation with homogeneous boudnary condition:
$$
\left\{\begin{array}{rl}
u' - \epsilon u'' = 0 &\quad \text{in} \ \Omega \\
u = 0 &\quad \text{on} \ \partial\Omega
\end{array}\right.
$$
where $\epsilon$ is a positive constant.

In [ ]:
def mesh_fem_1d(a,b,M,k):
    nrNodes = k*M + 1
    c4n = np.linspace(a, b, nrNodes)
    n4e = np.array([[i*k, (i+1)*k] for i in range(M)])
    n4db = np.array([0, nrNodes-1])
    ind4e = np.array([list(range(i*k, (i+1)*k+1)) for i in range(M)])
    return (c4n,n4e,n4db,ind4e)

In [ ]:
def get_matrices_1d(k=1):
    if k == 1:
        M_R = np.array([[2, 1],[1, 2]], dtype=np.float64) / 3.
        S_R = np.array([[1, -1],[-1, 1]], dtype=np.float64) / 2.
        D_R = np.array([[-1, 1],[-1, 1]], dtype=np.float64) / 2.
    elif k == 2:
        M_R = np.array([[4, 2, -1],[2, 16, 2],[-1, 2, 4]], dtype=np.float64) / 15.
        S_R = np.array([[7, -8, 1],[-8, 16, -8],[1, -8, 7]], dtype=np.float64) / 6.
        D_R = np.array([[-3, 4, -1],[-1, 0, 1],[1, -4, 3]], dtype=np.float64) / 2.

    ###################################################################################################
	# TODO: Add the matrices for the cubic approximations ($k=3$).

	###################################################################################################
    else:
        M_R = 0
        S_R = 0
        D_R = 0
    return (M_R, S_R, D_R)

**Q)** Complete the following code `fem_for_convection_diffusion_1d`

In [ ]:
def fem_for_convection_diffusion_1d(c4n,n4e,n4db,ind4e,k,M_R,S_R,D_R,f,u_D,epsilon):

    return

In [ ]:
a = 0
b = 1
T = 1
k = 1
epsilon = 1e-3

iter = 10
M = 2 ** np.arange(2,iter+2)
f = lambda x: 0*x + 1
u_D = lambda x: 0*x

M_R, S_R, D_R = get_matrices_1d(k)

for i in range(iter):
	c4n, n4e, n4db, ind4e = mesh_fem_1d(a,b,M[i],k)
	u = fem_for_convection_diffusion_1d(c4n,n4e,n4db,ind4e,k,M_R,S_R,D_R,f,u_D,epsilon)
	plt.figure()
	plt.plot(c4n,u)
	plt.title(f'$\epsilon$= {epsilon},\t h = 1/{M[i]}')

# 2D

Consider the following convection-diffusion equation with homogeneous boudnary condition:
$$
\left\{\begin{array}{rl}
\boldsymbol{a}\cdot\nabla u - \epsilon \Delta u = 0 &\quad \text{in} \ \Omega \\
u = 0 &\quad \text{on} \ \partial\Omega
\end{array}\right.
$$
where $\boldsymbol{a}$ is a convection vector and $\epsilon$ is a positive constant.

In [ ]:
def mesh_fem_2d(xl, xr, yl, yr, Mx, My, k):
	tmp = np.array([[np.arange(0,k*Mx,k) + (k*Mx+1)*i*k] for i in range(My)]).flatten()
	tmp2 = np.array([[np.arange(k,k*Mx+1,k) + (k*Mx+1)*(i+1)*k] for i in range(My)]).flatten()
	tmp3 = list(np.concatenate([list(j*(Mx*k+1)+np.arange(0,k+1-j)) for j in range(k+1)]))
	ind4e = np.array([tmp[np.int32(i/2)]+tmp3 if i % 2 == 0 else tmp2[np.int32((i-1)/2)] - tmp3 for i in range(2*Mx*My)])
	n4e = np.array([[ind4e[i,k], ind4e[i,np.int32((k+1)*(k+2)/2-1)], ind4e[i,0]] for i in range(ind4e.shape[0])])
	n4db = np.concatenate([[i for i in range(0,k*Mx+1)], [i for i in range(2*k*Mx+1,(k*Mx+1)*(k*My+1),(k*Mx+1))],
			[i for i in range((k*Mx+1)*(k*My+1)-2,k*My*(k*Mx+1)-1,-1)], [i for i in range((k*My-1)*(k*Mx+1),k*Mx,-(k*Mx+1))]])
	x = np.linspace(xl,xr,k*Mx+1)
	y = np.linspace(yl,yr,k*My+1)
	x = np.tile(x, (1,k*My+1)).flatten()
	y = np.tile(y,(k*Mx+1,1)).T.flatten()
	c4n = np.array([[x[i], y[i]] for i in range(len(x))])
	return (c4n,n4e,n4db,ind4e)

In [ ]:
def get_matrices_2d_triangle(k=1):
	if k == 1:
		M_R = np.array([[2, 1, 1], [1, 2, 1], [1, 1, 2]])/6.
		Srr_R = np.array([[1, -1, 0], [-1, 1, 0], [0, 0, 0]])/2.
		Srs_R = np.array([[1, 0, -1], [-1, 0, 1], [0, 0, 0]])/2.
		Ssr_R = np.array([[1, -1, 0], [0, 0, 0], [-1, 1, 0]])/2.
		Sss_R = np.array([[1, 0, -1], [0, 0, 0], [-1, 0, 1]])/2.
		Dr_R = np.array([[-1, 1, 0], [-1, 1, 0], [-1, 1, 0]])/2.
		Ds_R = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])/2.
	elif k == 2:
		M_R = np.array([[6, 0, -1, 0, -4, -1], [0, 32, 0, 16, 16, -4], [-1, 0, 6, -4, 0, -1],
			[0, 16, -4, 32, 16, 0], [-4, 16, 0, 16, 32, 0], [-1, -4, -1, 0, 0, 6]])/90.
		Srr_R = np.array([[3 ,-4, 1, 0, 0, 0], [-4, 8, -4, 0, 0, 0], [1, -4, 3, 0, 0, 0],
			[0, 0, 0, 8, -8, 0], [0, 0, 0, -8, 8, 0], [0, 0, 0, 0, 0, 0]])/6.
		Srs_R = np.array([[3, 0, 0, -4, 0, 1], [-4, 4, 0, 4, -4, 0], [1, -4, 0, 0, 4, -1],
			[0, 4, 0, 4, -4, -4], [0, -4, 0, -4, 4, 4], [0, 0, 0, 0, 0, 0]])/6.
		Ssr_R = np.array([[3, -4, 1, 0, 0, 0], [0, 4, -4, 4, -4, 0], [0, 0, 0, 0, 0, 0],
			[-4, 4, 0, 4, -4, 0], [0, -4, 4, -4, 4, 0], [1, 0, -1, -4, 4, 0]])/6.
		Sss_R = np.array([[3, 0, 0, -4, 0, 1], [0, 8, 0, 0, -8, 0], [0, 0, 0, 0, 0, 0],
			[-4, 0, 0, 8, 0, -4], [0, -8, 0, 0, 8, 0], [1, 0, 0, -4, 0, 3]])/6.
		Dr_R = np.array([[-3, 4, -1, 0, 0, 0], [-1, 0, 1, 0, 0, 0], [1, -4, 3, 0, 0, 0],
			[-1, 2, -1, -2, 2, 0], [1, -2, 1, -2, 2, 0], [1, 0, -1, -4, 4, 0]])/2.
		Ds_R = np.array([[-3, 0, 0, 4, 0, -1], [-1, -2, 0, 2, 2, -1], [1, -4, 0, 0, 4, -1],
			[-1, 0, 0, 0, 0, 1], [1, -2, 0, -2, 2, 1], [1, 0, 0, -4, 0, 3]])/2.

	###################################################################################################
	# TODO: Add the matrices for the cubic approximations ($k=3$).

	###################################################################################################
	else:
		M_R = 0
		Srr_R = 0
		Srs_R = 0
		Ssr_R = 0
		Sss_R = 0
		Dr_R = 0
		Ds_R = 0
	return (M_R, Srr_R, Srs_R, Ssr_R, Sss_R, Dr_R, Ds_R)

**Q)** Complete the following code `fem_for_convection_diffusion_2d`

In [ ]:
def fem_for_convection_diffusion_2d(c4n,n4e,n4db,ind4e,M_R,Srr_R,Srs_R,Ssr_R,Sss_R,Dr_R,Ds_R,f,a,epsilon,u_D):

    return

In [ ]:
iter = 6
xl, xr, yl, yr=0, 1, 0, 1
M = 2 ** np.arange(2,iter+2)
f = lambda x: 0 * x[:,0] + 1
u_D = lambda x: 0 * x[:,0]
a = np.array([3,1])/np.sqrt(10)
epsilon = 1e-4

h = 1 / M
k = 1
error = np.zeros(iter)
M_R, Srr_R, Srs_R, Ssr_R, Sss_R, Dr_R, Ds_R = get_matrices_2d_triangle(k)

fig = plt.figure(figsize=(10, 6*iter))
for i in range(iter):
    c4n, n4e, n4db, ind4e = mesh_fem_2d(xl, xr, yl, yr, M[i], M[i], k)
    u = fem_for_convection_diffusion_2d(c4n,n4e,n4db,ind4e,M_R,Srr_R,Srs_R,Ssr_R,Sss_R,Dr_R,Ds_R,f,a,epsilon,u_D)
    ax = fig.add_subplot(iter, 1, i + 1, projection='3d')
    ax.plot_trisurf(c4n[:, 0], c4n[:, 1], u, triangles=n4e, cmap='viridis')
    ax.set_title(f'FE solution with h = 1/{M[i]}')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.set_zlim([-.5,1.25])
    ax.view_init(elev=15, azim= -110)